# 2DEG mobility calculation using analytical formula

Ref-1: J. Bassaler, J. Mehta, I. Abid, L. Konczewicz, S. Juillaguet, S. Contreras, S. Rennesson, S. Tamariz, M. Nemoz, F. Semond, J. Pernot, F. Medjdoub, Y. Cordier, P. Ferrandis, Al-Rich AlGaN Channel High Electron Mobility Transistors on Silicon: A Relevant Approach for High Temperature Stability of Electron Mobility. Adv. Electron. Mater. 2024, 2400069. https://doi.org/10.1002/aelm.202400069

Ref-2: Zhang, J., Hao, Y., Zhang, J. et al. The mobility of two-dimensional electron gas in AlGaN/GaN heterostructures with varied Al content. Sci. China Ser. F-Inf. Sci. 51, 780–789 (2008). https://doi.org/10.1007/s11432-008-0056-7


# 1. Settings

## 1.1 Import modules

In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import colors, cm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
from pathlib import Path
mobilitypy_tutorial_path = Path().absolute()
sys.path.insert(0, os.path.join(mobilitypy_tutorial_path, '../'))
from mobilitypy import AlloyParams, Mobility2DEG, Plottings, PlotQuasi3DFuns

In [4]:
save_figure_dir = '/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/MobilityCalc2DEG/OUTPUTS/FIGS'
savefigure=True

In [5]:
folder_post_data_ = '/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/NextnanoSimulations/DATAs/AlGaN_HEMT/AlGaN_AlGaN_AlN_HEMT/nnp/2DEHG_density/OneDim/'
data_file = 'sim_post_process_data.xlsx'
excel_file = f'{folder_post_data_}/{data_file}' 

## 1.2 Set physical constants

In [6]:
'''
n_2d => in nm^-2
rms_roughness => nm^-1
corr_len => nm^-1
n_dis => nm^-2
f_dis => unit less
'''
alloy_disordered_effect=1
interface_roughness_effect=1
dislocation_effect=1
deformation_potential_effect=1 
piezoelectric_effect=1
acoustic_phonon_effect=1
polar_optical_phonon_effect=1
total_mobility=1
mobility_model='Bassaler'
density_2deg = 0.1 # nm^-2
irf_rms_roughness = 0.3 # nm
irf_corr_length = 3.0 # nm
dislocation_density = 1e-4 # nm^-2
occup_dislocation = 0.3
T=300 #K

In [7]:
plot_mobilities = ['IFR', 'DIS', 'AD', 'AP', 'POP', 'TOT', 'LFOM', 'LFOMnorm', '2DEG']
len_plots = len(plot_mobilities)
rescale_2deg_fact=1e13

In [8]:
params = {'axes.labelsize': 20,'axes.titlesize': 20,
          'xtick.labelsize':20,
                  'xtick.major.width':2,
                  'xtick.major.size':5,
                  'xtick.minor.width':2,
                  'xtick.minor.size':3,
                  'ytick.labelsize': 20,
                  'ytick.major.width':2,
                  'ytick.major.size':5,
                  'ytick.minor.width':2,
                  'ytick.minor.size':3,
                  'errorbar.capsize':2}
plt.rcParams.update(params)
plt.rc('font', size=20)

# 2. Calculate mobilities

## 2.1 GaN channel LFOM for normalization

### 2.1.1 Read 2DEG data from nextnano simulation

In [9]:
data_sheet_name =  'GaN_C_sweep__AlContentBarrier__ThicknessAlGaNBarrier'
n_2d_df_ref = pd.read_excel(excel_file, sheet_name=data_sheet_name, index_col=0)
density_2deg_comp_ref = np.array(n_2d_df_ref['2DEG_device'], dtype=float)/1e+14 # cm^-2 to nm^-2 conversion
channel_comp_ref = np.zeros(len(density_2deg_comp_ref))

### 2.1.2 Calculate AlGaN/GaN mobilities

In [10]:
mu2deg = Mobility2DEG(compositions=channel_comp_ref, binaries=['AlN', 'GaN'], alloy='AlGaN', system='ternary', 
                      print_log=None, eps_n_2d=1e-10)

In [11]:
mobility_ref = mu2deg.calculate_mobility(n_2d=density_2deg_comp_ref, rms_roughness=irf_rms_roughness, 
                                        corr_len=irf_corr_length, n_dis=dislocation_density, 
                                        f_dis=occup_dislocation, T=T,
                                        alloy_disordered_effect=False,
                                        interface_roughness_effect=interface_roughness_effect,
                                        dislocation_effect=dislocation_effect,
                                        deformation_potential_effect=deformation_potential_effect,
                                        piezoelectric_effect=piezoelectric_effect,
                                        acoustic_phonon_effect=acoustic_phonon_effect,
                                        polar_optical_phonon_effect=polar_optical_phonon_effect,
                                        total_mobility=total_mobility,
                                        mobility_model=mobility_model)

### 2.1.3 Calculate AlGaN/GaN device figure-of-merits

In [12]:
mobility_ref['LFOM'] = mu2deg.calculate_figure_of_merit(density_2deg_comp_ref, np.array(mobility_ref['TOT']))
Combined_df_ref = pd.concat([n_2d_df_ref, mobility_ref], axis=1)

In [13]:
print('- AlGaN/GaN with highest LFOM (MW/cm^2):\n')
Combined_df_ref.loc[[Combined_df_ref['LFOM'].idxmax()]]

- AlGaN/GaN with highest LFOM (MW/cm^2):



,AlContentBarrier,ThicknessAlGaNBarrier,2DEG_device,2DEG_BC,2DEG_SC,2DHG_device,2DHG_BC,2DHG_SC,comp,IFR,DIS,DP,PE,AP,POP,TOT,LFOM
189,1.0,50,5.234338e+13,5.234338e+13,1.076399e-41,55093120000000,2.862996e-43,55093119000000,0.000,18748.949898,523781.628432,4629.583066,54305.7741,4265.91276,1297.054369,942.827294,112348.462817


In [14]:
ref_LFOM = Combined_df_ref['LFOM'].max()
print(f'- AlGaN/GaN with highest LFOM = {ref_LFOM:.5e} MW/cm^2')

- AlGaN/GaN with highest LFOM = 1.12348e+05 MW/cm^2


### 2.1.4 Alternative refrence AlGaN/GaN device

o This is a AlGaN barrier-GaN channel device with barrier AlGaN composition and barrier thickness such that 2DEG density=1x10^13 /cm^2

In [15]:
mu2deg = Mobility2DEG(compositions=0, binaries=['AlN', 'GaN'], alloy='AlGaN', system='ternary')
mobility_ref_a = mu2deg.calculate_mobility(n_2d=0.1, rms_roughness=irf_rms_roughness, 
                                        corr_len=irf_corr_length, n_dis=dislocation_density, 
                                        f_dis=occup_dislocation, T=T,
                                        interface_roughness_effect=interface_roughness_effect,
                                        dislocation_effect=dislocation_effect,
                                        deformation_potential_effect=deformation_potential_effect,
                                        piezoelectric_effect=piezoelectric_effect,
                                        acoustic_phonon_effect=acoustic_phonon_effect,
                                        polar_optical_phonon_effect=polar_optical_phonon_effect,
                                        total_mobility=total_mobility,
                                        mobility_model=mobility_model)
ref_LFOM_a = mu2deg.calculate_figure_of_merit(0.1, np.array(mobility_ref_a['TOT']))[0]
print(f'- This specific AlGaN/GaN has LFOM = {ref_LFOM_a:.5e} MW/cm^2')

- This specific AlGaN/GaN has LFOM = 3.16497e+04 MW/cm^2


### 2.1.5 Normalized AlGaN/GaN device

In [16]:
Combined_df_ref['LFOMnorm'] = Combined_df_ref['LFOM'].copy()/ref_LFOM

In [17]:
Combined_df_ref

,AlContentBarrier,ThicknessAlGaNBarrier,2DEG_device,2DEG_BC,2DEG_SC,2DHG_device,2DHG_BC,2DHG_SC,comp,IFR,DIS,DP,PE,AP,POP,TOT,LFOM,LFOMnorm
0,0.1,5,2.618620e-05,2.618579e-05,3.618691e-43,55267126000000,4.069552e-27,55267120000000,0.000,NaN,3.822296e-14,NaN,NaN,NaN,NaN,3.822296e-14,2.278610e-30,2.028163e-35
1,0.1,10,1.681836e+00,1.681792e+00,9.360279e-43,55217285000000,6.271308e-30,55217282000000,0.000,NaN,2.454912e-09,NaN,NaN,NaN,NaN,2.454912e-09,9.399242e-21,8.366151e-26
2,0.1,15,7.689244e+04,7.689176e+04,2.390214e-42,55169079000000,9.660822e-35,55169078000000,0.000,4.834625e+26,1.122591e-04,1.569811e+14,9.929912e+06,9.929911e+06,1.110991e+06,1.122591e-04,1.965072e-11,1.749087e-16
3,0.1,20,2.712707e+09,2.712699e+09,5.979782e-42,55122617000000,2.007350e-39,55122616000000,0.000,5.784175e+12,4.065894e+00,4.602133e+07,3.288230e+04,3.285883e+04,3.402420e+04,4.064906e+00,2.510305e-02,2.234392e-07
4,0.1,25,3.142068e+11,3.142065e+11,8.653905e-42,55104046000000,4.136819e-42,55104045000000,0.000,5.065016e+06,6.416729e+02,8.952478e+04,6.375812e+03,5.951925e+03,7.300236e+03,5.365905e+02,3.838237e+02,3.416368e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,1.0,30,5.055013e+13,5.055012e+13,1.075161e-41,55093178000000,2.896870e-43,55093176000000,0.000,1.898060e+04,4.980135e+05,4.690574e+03,5.308460e+04,4.309762e+03,1.305105e+03,9.497157e+02,1.092922e+05,9.727964e-01
186,1.0,35,5.118690e+13,5.118690e+13,1.075606e-41,55093157000000,2.884641e-43,55093156000000,0.000,1.889724e+04,5.071153e+05,4.668562e+03,5.351959e+04,4.293993e+03,1.302195e+03,9.472324e+02,1.103795e+05,9.824748e-01
187,1.0,40,5.166716e+13,5.166716e+13,1.075938e-41,55093141000000,2.875562e-43,55093140000000,0.000,1.883518e+04,5.140152e+05,4.652222e+03,5.384667e+04,4.282246e+03,1.300038e+03,9.453869e+02,1.111981e+05,9.897608e-01
188,1.0,45,5.204229e+13,5.204228e+13,1.076194e-41,55093129000000,2.868560e-43,55093128000000,0.000,1.878718e+04,5.194256e+05,4.639611e+03,5.410156e+04,4.273156e+03,1.298375e+03,9.439614e+02,1.118366e+05,9.954437e-01


## 2.2 Plotting reference AlGaN/GaN HEMT properties

In [18]:
lpltq3d = PlotQuasi3DFuns(save_figure_dir=f'{save_figure_dir}/2DEG_mobilities_ref')

In [19]:
Combined_df_ref['2DEG'] =  Combined_df_ref['2DEG_device'].copy()/rescale_2deg_fact
Combined_df_ref['2DHG'] =  Combined_df_ref['2DHG_device'].copy()/rescale_2deg_fact

In [20]:
fig_ncols, fig_nrows = 3, 3
fig, axs = plt.subplots(fig_ncols, fig_nrows, figsize=(16,12), 
                        sharey=True,constrained_layout=True)
show_colorbar = False
tick_multiplicator=[0.2, 0.1, 10, 5]
for ii in range(fig_ncols*fig_nrows):
    if ii < len(plot_mobilities):
        which_mobility = plot_mobilities[ii]
        if which_mobility not in Combined_df_ref: 
            #axs[ii//3][ii%3].axis("off") 
            axs[ii//3][ii%3].get_yaxis().set_visible(False)
            axs[ii//3][ii%3].get_xaxis().set_visible(False)
            axs[ii//3][ii%3].set_title(which_mobility)
            continue
    else:
        axs[ii//3][ii%3].axis("off") 
        continue
    x_label_text, y_label_text, z_label = '','',''
    print(f'\t-- Plotting {which_mobility}...')
    xx, yy, zz = Combined_df_ref['AlContentBarrier'], Combined_df_ref['ThicknessAlGaNBarrier'], Combined_df_ref[which_mobility]

    vmin = 1E-5 if which_mobility in ['2DEG', 'LFOM', 'LFOMnorm'] else 1 #Combined_df_ref[which_mobility].min(numeric_only = True)
    vmax = min(Combined_df_ref[which_mobility].max(numeric_only = True), 1e10)
    #vmin, vmax = zz.min(numeric_only = True)+1e-20, zz.max(numeric_only = True)
    norm = colors.Normalize(vmin=vmin, vmax=vmax) if which_mobility in ['2DEG', 'LFOMnorm'] else colors.LogNorm(vmin=vmin, vmax=vmax)
    cbar_mapable = cm.ScalarMappable(norm=norm, cmap='viridis')
    if ii-6>=0: x_label_text = 'Al composition barrier'
    if ii%3==0: y_label_text = 'Barrier thickness (nm)'
    if ii%3-2==0: z_label=r'$\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)'
    if which_mobility == 'LFOM': z_label=r'LFOM (MW/cm$^2$)'
    if which_mobility == 'LFOMnorm': z_label=r'LFOM$_\mathrm{AlGaN}$/LFOM$_\mathrm{highest}$'
    if which_mobility == '2DEG': z_label=r'2DEG ($\mathrm{10}^{13}\mathrm{cm}^{-2}$)'

    ##### Plot composition map 
    _, _ = lpltq3d.Plotq3D(xx,yy,zz, fig=fig, ax=axs[ii//3][ii%3], 
                           x_label=x_label_text, y_label=y_label_text,
                           z_label=z_label, interpolation_method='linear',
                           interpolation_points = 20,
                           tick_multiplicator=tick_multiplicator,
                           title_label=which_mobility, norm=norm,
                           cbar_mappable=cbar_mapable, show_contour_lines=1,
                           marker='s', marker_size=24**2,
                           cbar_text=z_label, show_colorbar=1,
                           plot_controur=1, plot_scatter=0, 
                           savefigure=False, show_plot=False)
    if ii%3 != 0: axs[ii//3][ii%3].get_yaxis().set_visible(False)
lpltq3d.save_figure(f'GaNChannel_AlGaN.png', savefig=savefigure, show_plot=True,
                    fig=fig, CountFig=None, dpi=75)
print('- Done', flush=True)
#break

	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...


/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/MobilityCalc2DEG/mobilitypy/tutorials/../mobilitypy/utilities/_quasi3d_plot_fns.py:56: UserWarning: Log scale: values of z <= 0 have been masked
  CS = axs.contourf(xi, yi, zi, cmap=color_map, norm=norm)


- Done


## 2.3 AlGaN/AlGaN mobilities and figure-of-merit

### 2.3.1 Read 2DEG data from nextnano simulation

In [21]:
data_sheet_name =  'sim_sweep__AlContentBarrier__AlContentChannel__ThicknessAlGaNBarrier'
n_2d_df = pd.read_excel(excel_file, sheet_name=data_sheet_name, index_col=0)
density_2deg_comp = np.array(n_2d_df['2DEG_device'], dtype=float)/1e+14 # cm^-2 to nm^-2 conversion
channel_comp = np.array(n_2d_df['AlContentChannel'], dtype=float)

### 2.3.2 Calculate mobilities

In [22]:
mu2deg = Mobility2DEG(compositions=channel_comp, binaries=['AlN', 'GaN'], alloy='AlGaN', system='ternary')

In [23]:
mobility_df = mu2deg.calculate_mobility(n_2d=density_2deg_comp, rms_roughness=irf_rms_roughness, 
                                        corr_len=irf_corr_length, n_dis=dislocation_density, 
                                        f_dis=occup_dislocation, T=T,
                                        alloy_disordered_effect=alloy_disordered_effect,
                                        interface_roughness_effect=interface_roughness_effect,
                                        dislocation_effect=dislocation_effect,
                                        deformation_potential_effect=deformation_potential_effect,
                                        piezoelectric_effect=piezoelectric_effect,
                                        acoustic_phonon_effect=acoustic_phonon_effect,
                                        polar_optical_phonon_effect=polar_optical_phonon_effect,
                                        total_mobility=total_mobility,
                                        mobility_model=mobility_model)

### 2.3.3 Calculate AlGaN lateral figure-of-merit

In [24]:
mobility_df['LFOM'] = mu2deg.calculate_figure_of_merit(density_2deg_comp, np.array(mobility_df['TOT']))
mobility_df['LFOMnorm'] = mobility_df['LFOM'].copy()/ref_LFOM

### 2.3.3 Combine AlGaN mobilities and lateral figure-of-merit dataframe

In [25]:
Combined_df = pd.concat([n_2d_df, mobility_df], axis=1)

## 2.4 Plot AlGaN/AlGaN HEMT mobilities

In [26]:
lpltq3d = PlotQuasi3DFuns(save_figure_dir=f'{save_figure_dir}/2DEG_mobilities')

In [27]:
Combined_df['2DEG'] =  Combined_df['2DEG_device'].copy()/rescale_2deg_fact
Combined_df['2DHG'] =  Combined_df['2DHG_device'].copy()/rescale_2deg_fact

In [28]:
df_cap_thickness_group = Combined_df.groupby(['ThicknessAlGaNBarrier']) # Group by thickness

fig_ncols, fig_nrows = 3, 3
for name, group in df_cap_thickness_group:
    barrier_thickness = f'{name[0]:.2f}'
    print(f'- Plotting barrier thickness = {barrier_thickness} nm')
    fig, axs = plt.subplots(fig_ncols, fig_nrows, figsize=(16,13.2), 
                            sharey=True,constrained_layout=True)
    show_colorbar = False
    tick_multiplicator=[0.1, 0.05, 0.1, 0.05]
    for ii in range(fig_ncols*fig_nrows):
        if ii < len(plot_mobilities):
            which_mobility = plot_mobilities[ii]
        else:
            axs[ii//3][ii%3].axis("off") 
            continue
        x_label_text, y_label_text, z_label = '','',''
        print(f'\t-- Plotting {which_mobility}...')
        xx, yy, zz = group['AlContentChannel'], group['AlContentBarrier'], group[which_mobility]

        vmin = 1e-5 if which_mobility in ['2DEG', 'LFOM', 'LFOMnorm'] else 1
        vmax = min(Combined_df[which_mobility].max(numeric_only=True), 1e10)
        #vmin, vmax = zz.min()+1e-20, zz.max()
        norm = colors.Normalize(vmin=vmin, vmax=vmax) if which_mobility in ['LFOMnorm','2DEG'] else colors.LogNorm(vmin=vmin, vmax=vmax)
        cbar_mapable = cm.ScalarMappable(norm=norm, cmap='viridis')
        #print(ii, ii%3)
        if ii-6>=0: x_label_text = 'Al composition channel'
        if ii%3==0: y_label_text = 'Al composition barrier'
        if ii%3-2==0: z_label=r'$\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)'
        if which_mobility == 'LFOM': z_label=r'LFOM (MW/cm$^2$)'
        if which_mobility == 'LFOMnorm': z_label=r'LFOM$_\mathrm{AlGaN}$/LFOM$_\mathrm{GaN}$'
        if which_mobility == '2DEG': z_label=r'2DEG ($\mathrm{10}^{13}\mathrm{cm}^{-2}$)'

        ##### Plot composition map 
        fig, _ = lpltq3d.Plotq3D(xx,yy,zz, fig=fig, ax=axs[ii//3][ii%3],
                                 vmin=vmin, vmax=vmax, xmin=0.475, xmax=0.975, ymin=0.525, ymax=1.025,
                                                x_label=x_label_text, y_label=y_label_text,
                                                z_label=z_label, interpolation_method='linear',
                                                interpolation_points = 20,
                                                tick_multiplicator=tick_multiplicator,
                                                title_label=which_mobility[:], 
                                                cbar_mappable=cbar_mapable, norm=norm,
                                                show_contour_lines=False, 
                                                marker='s', marker_size=24**2,
                                                cbar_text=z_label,show_colorbar=True,
                                                plot_controur=0, plot_scatter=1, 
                                                savefigure=False, show_plot=False)
        plt.suptitle(f"Barrier thickness = {barrier_thickness} nm", size=24)
        if ii%3 != 0: axs[ii//3][ii%3].get_yaxis().set_visible(False)
    lpltq3d.save_figure(f'Barrier_{barrier_thickness}.png', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=75)
    print('- Done', flush=True)
    #break

- Plotting barrier thickness = 5.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 10.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 15.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 20.00 nm
	-- Plotting IFR...
	-- Plotting DIS...
	-- Plotting AD...
	-- Plotting AP...
	-- Plotting POP...
	-- Plotting TOT...
	-- Plotting LFOM...
	-- Plotting LFOMnorm...
	-- Plotting 2DEG...
- Done
- Plotting barrier thickness = 25.00 nm
	-- Plotting IFR...
	-- Plotting 

## 2.5 Plot 2D plots

In [29]:
plt2deg = Plottings(save_figure_dir=f'{save_figure_dir}/Others')

### 2.5.1 Mobility plot

In [30]:
x_label = 'Barrier thickness (nm)'
y_label = r'Electron mobility ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)'
save_file_name_ = 'mu_Tb_AlNB_AlGaNC.png'
tick_multiplicator=[10, 5, 50, 25]

In [31]:
Combined_df_ = Combined_df[Combined_df['AlContentBarrier']>0.99]
df_cap_thickness_group = Combined_df_.groupby(['AlContentChannel']) # Group by thickness

fig, axs = plt.subplots(figsize=(9,6),constrained_layout=True)
for name, group in df_cap_thickness_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['TOT']
    axs.plot(xx, zz, 'o-',label=f'{name[0]:.2f}')
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=75, show_plot=False)

### 2.5.1 FOM plot

In [32]:
x_label = 'Barrier thickness (nm)'
y_label = r'LFOM$_\mathrm{AlGaN}$/LFOM$_\mathrm{GaN}$'
save_file_name_ = 'lfom_Tb_AlNB_AlGaNC.png'
tick_multiplicator=[10, 5, 0.05, 0.025]

In [33]:
# Combined_df_ = Combined_df[Combined_df['AlContentBarrier']>0.99]
# df_cap_thickness_group = Combined_df_.groupby(['AlContentChannel']) # Group by thickness

fig, axs = plt.subplots(figsize=(9,6),constrained_layout=True)
for name, group in df_cap_thickness_group:
    xx, zz = group['ThicknessAlGaNBarrier'],  group['LFOMnorm']
    axs.plot(xx, zz, 'o-',label=f'{name[0]:.2f}')
#axs.set_yscale('log')
axs.set_xlabel(x_label)
axs.set_ylabel(y_label)
axs.legend(bbox_to_anchor=(1, 0.5),loc='center left') 
plt2deg.set_tickers(axs, tick_multiplicator)
plt2deg.save_figure(save_file_name_, fig=fig, savefig=savefigure, dpi=75, show_plot=False)